<a href="https://colab.research.google.com/github/swilsonmfc/pandas/blob/main/3_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework Lesson 3

# Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
from pprint import pprint as pp

# Data

## Download the Data

In [ ]:
!wget https://www.postgresqltutorial.com/wp-content/uploads/2019/05/dvdrental.zip
!unzip dvdrental.zip
!tar -xvf dvdrental.tar
!rm toc.dat

--2021-04-25 18:56:31--  https://www.postgresqltutorial.com/wp-content/uploads/2019/05/dvdrental.zip
Resolving www.postgresqltutorial.com (www.postgresqltutorial.com)... 151.139.128.11
Connecting to www.postgresqltutorial.com (www.postgresqltutorial.com)|151.139.128.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 550906 (538K) [application/zip]
Saving to: ‘dvdrental.zip’

dvdrental.zip       100%[===================>] 537.99K  --.-KB/s    in 0.03s   

2021-04-25 18:56:31 (20.6 MB/s) - ‘dvdrental.zip’ saved [550906/550906]

Archive:  dvdrental.zip
  inflating: dvdrental.tar           
toc.dat
3057.dat
3065.dat
3059.dat
3067.dat
3069.dat
3055.dat
3061.dat
3062.dat
3063.dat
3071.dat
3073.dat
3075.dat
3077.dat
3079.dat
3081.dat
restore.sql


## Friendly Tables & Columns

In [ ]:
table_name_map = {
  '3057.dat':'actor',
  '3065.dat':'address',
  '3059.dat':'category',
  '3067.dat':'city',
  '3069.dat':'country',
  '3055.dat':'customer',
  '3061.dat':'film',
  '3062.dat':'film_actor',
  '3063.dat':'film_category',
  '3071.dat':'inventory',
  '3073.dat':'language',
  '3075.dat':'payment',
  '3077.dat':'rental',
  '3079.dat':'staff',
  '3081.dat':'store'
}

In [ ]:
table_column_map = {
  '3057.dat':['actor_id', 'first_name', 'last_name', 'last_update'],
  '3065.dat':['address_id', 'address', 'address2', 'district', 'city_id', 'postal_code', 'phone', 'last_update'],
  '3059.dat':['category_id', 'name', 'last_update'],
  '3067.dat':['city_id', 'city', 'country_id', 'last_update'],
  '3069.dat':['country_id', 'country', 'last_update'],
  '3055.dat':['customer_id', 'store_id', 'first_name', 'last_name', 'email', 'address_id', 'activebool', 'create_date', 'last_update', 'active'],
  '3061.dat':['film_id', 'title', 'description', 'release_year', 'language_id', 'rental_duration', 'rental_rate', 'length', 'replacement_cost', 'rating', 'last_update', 'special_features', 'fulltext'],
  '3062.dat':['actor_id', 'film_id', 'last_update'],
  '3063.dat':['film_id', 'category_id', 'last_update'],
  '3071.dat':['inventory_id', 'film_id', 'store_id', 'last_update'],
  '3073.dat':['language_id', 'name', 'last_update'],
  '3075.dat':['payment_id', 'customer_id', 'staff_id', 'rental_id', 'amount', 'payment_date'],
  '3077.dat':['rental_id', 'rental_date', 'inventory_id', 'customer_id', 'return_date', 'staff_id', 'last_update'],
  '3079.dat':['staff_id', 'first_name', 'last_name', 'address_id', 'email', 'store_id', 'active', 'username', 'password', 'last_update', 'picture'],
  '3081.dat':['store_id', 'manager_staff_id', 'address_id', 'last_update']
}

## Load Data

In [ ]:
# Load each .dat file as a pandas frame adding column headers
data = {str(file): pd.read_csv(file, sep='\t', names=table_column_map[str(file)]) \
      for file in pathlib.Path('.').rglob('*.dat')}

# Remove last row of each table (drop takes the indexes to remove)
data = {name: table.drop(table.tail(1).index) for name, table in data.items()}

# Rename keys for friendly names 
db = {table_name_map[key]:data.pop(key) for key, val in table_name_map.items()}

# Change all id columns to int (rather than floats)
for name, table in db.items():
  for col in table.columns:
    if col.endswith('_id'):
      table[col] = table[col].astype('int') 

pp(db)

{'actor':      actor_id first_name     last_name             last_update
0           1   Penelope       Guiness  2013-05-26 14:47:57.62
1           2       Nick      Wahlberg  2013-05-26 14:47:57.62
2           3         Ed         Chase  2013-05-26 14:47:57.62
3           4   Jennifer         Davis  2013-05-26 14:47:57.62
4           5     Johnny  Lollobrigida  2013-05-26 14:47:57.62
..        ...        ...           ...                     ...
195       196       Bela        Walken  2013-05-26 14:47:57.62
196       197      Reese          West  2013-05-26 14:47:57.62
197       198       Mary        Keitel  2013-05-26 14:47:57.62
198       199      Julia       Fawcett  2013-05-26 14:47:57.62
199       200      Thora        Temple  2013-05-26 14:47:57.62

[200 rows x 4 columns],
 'address':      address_id                   address  ...         phone          last_update
0             1         47 MySakila Drive  ...           NaN  2006-02-15 09:45:30
1             2        28 MySQL B

## Schema
![](https://www.postgresqltutorial.com/wp-content/uploads/2018/03/dvd-rental-sample-database-diagram.png)

# Where do you work?
* Example of the one-many join

In [ ]:
# How many staff are there?
len(db['staff'])

2

In [ ]:
# How many stores are there?
len(db['store'])

2

In [ ]:
# How much staff work at each store?
staffing_df = db['store'].merge(db['staff'], on='store_id')
staffing_df.groupby('store_id').size()

store_id
1    1
2    1
dtype: int64

# Who's in what film?
* Multi DataFrame join

In [ ]:
# Print the Actors Table
db['actor']

,actor_id,first_name,last_name,last_update
0,1,Penelope,Guiness,2013-05-26 14:47:57.62
1,2,Nick,Wahlberg,2013-05-26 14:47:57.62
2,3,Ed,Chase,2013-05-26 14:47:57.62
3,4,Jennifer,Davis,2013-05-26 14:47:57.62
4,5,Johnny,Lollobrigida,2013-05-26 14:47:57.62
...,...,...,...,...
195,196,Bela,Walken,2013-05-26 14:47:57.62
196,197,Reese,West,2013-05-26 14:47:57.62
197,198,Mary,Keitel,2013-05-26 14:47:57.62
198,199,Julia,Fawcett,2013-05-26 14:47:57.62


In [ ]:
# Print the film table
db['film']

,film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext
0,133,Chamber Italian,A Fateful Reflection of a Moose And a Husband ...,2006.0,1,7.0,4.99,117.0,14.99,NC-17,2013-05-26 14:50:58.951,{Trailers},'chamber':1 'fate':4 'husband':11 'italian':2 ...
1,384,Grosse Wonderful,A Epic Drama of a Cat And a Explorer who must ...,2006.0,1,5.0,4.99,49.0,19.99,R,2013-05-26 14:50:58.951,"{""Behind the Scenes""}",'australia':18 'cat':8 'drama':5 'epic':4 'exp...
2,8,Airport Pollock,A Epic Tale of a Moose And a Girl who must Con...,2006.0,1,6.0,4.99,54.0,15.99,R,2013-05-26 14:50:58.951,{Trailers},'airport':1 'ancient':18 'confront':14 'epic':...
3,98,Bright Encounters,A Fateful Yarn of a Lumberjack And a Feminist ...,2006.0,1,4.0,4.99,73.0,12.99,PG-13,2013-05-26 14:50:58.951,{Trailers},'boat':20 'bright':1 'conquer':14 'encount':2 ...
4,1,Academy Dinosaur,A Epic Drama of a Feminist And a Mad Scientist...,2006.0,1,6.0,0.99,86.0,20.99,PG,2013-05-26 14:50:58.951,"{""Deleted Scenes"",""Behind the Scenes""}",'academi':1 'battl':15 'canadian':20 'dinosaur...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Young Language,A Unbelieveable Yarn of a Boat And a Database ...,2006.0,1,6.0,0.99,183.0,9.99,G,2013-05-26 14:50:58.951,"{Trailers,""Behind the Scenes""}",'administr':12 'boat':8 'boy':17 'databas':11 ...
996,997,Youth Kick,A Touching Drama of a Teacher And a Cat who mu...,2006.0,1,4.0,0.99,179.0,14.99,NC-17,2013-05-26 14:50:58.951,"{Trailers,""Behind the Scenes""}",'boat':22 'cat':11 'challeng':14 'drama':5 'ki...
997,998,Zhivago Core,A Fateful Yarn of a Composer And a Man who mus...,2006.0,1,6.0,0.99,105.0,10.99,NC-17,2013-05-26 14:50:58.951,"{""Deleted Scenes""}",'boy':16 'canadian':19 'compos':8 'core':2 'fa...
998,999,Zoolander Fiction,A Fateful Reflection of a Waitress And a Boat ...,2006.0,1,5.0,2.99,101.0,28.99,R,2013-05-26 14:50:58.951,"{Trailers,""Deleted Scenes""}",'ancient':19 'boat':11 'china':20 'discov':14 ...


In [ ]:
# How many films are there?
len(db['film'])

1000

In [ ]:
# Set an index on the actor table using its id
db['actor'] = db['actor'].set_index('actor_id')

In [ ]:
# Set an index on the film table using its id
db['film'] = db['film'].set_index('film_id')

In [ ]:
# Set a multi-index on the film_actor table using both film_id and actor_id
db['film_actor'] = db['film_actor'].set_index(['actor_id', 'film_id'])

In [ ]:
# Which actors are in which films
actor_film_df = db['actor'].merge(db['film_actor'], left_index=True,   right_on='actor_id', how='inner') \
                           .merge(db['film'],       left_on='film_id', right_index=True,    how='inner')

In [ ]:
# Show the last 5 items of the actor_film_df
actor_film_df[-5:]

,,first_name,last_name,last_update_x,last_update_y,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext
actor_id,film_id,,,,,,,,,,,,,,,,
174,352,Michael,Bening,2013-05-26 14:47:57.62,2006-02-15 10:05:03,Gathering Calendar,A Intrepid Tale of a Pioneer And a Moose who m...,2006.0,1,4.0,0.99,176.0,22.99,PG-13,2013-05-26 14:50:58.951,"{Commentaries,""Behind the Scenes""}",'calendar':2 'conquer':14 'convent':20 'frisbe...
193,352,Burt,Temple,2013-05-26 14:47:57.62,2006-02-15 10:05:03,Gathering Calendar,A Intrepid Tale of a Pioneer And a Moose who m...,2006.0,1,4.0,0.99,176.0,22.99,PG-13,2013-05-26 14:50:58.951,"{Commentaries,""Behind the Scenes""}",'calendar':2 'conquer':14 'convent':20 'frisbe...
194,352,Meryl,Allen,2013-05-26 14:47:57.62,2006-02-15 10:05:03,Gathering Calendar,A Intrepid Tale of a Pioneer And a Moose who m...,2006.0,1,4.0,0.99,176.0,22.99,PG-13,2013-05-26 14:50:58.951,"{Commentaries,""Behind the Scenes""}",'calendar':2 'conquer':14 'convent':20 'frisbe...
197,995,Reese,West,2013-05-26 14:47:57.62,2006-02-15 10:05:03,Yentl Idaho,A Amazing Display of a Robot And a Astronaut w...,2006.0,1,5.0,4.99,86.0,11.99,R,2013-05-26 14:50:58.951,"{Trailers,Commentaries,""Deleted Scenes""}",'amaz':4 'astronaut':11 'berlin':18 'display':...
198,264,Mary,Keitel,2013-05-26 14:47:57.62,2006-02-15 10:05:03,Dwarfs Alter,A Emotional Yarn of a Girl And a Dog who must ...,2006.0,1,6.0,2.99,101.0,13.99,G,2013-05-26 14:50:58.951,"{Commentaries,""Deleted Scenes""}",'alter':2 'ancient':18 'challeng':14 'compos':...


In [ ]:
# What's the number of actors in each film?
actor_film_df.groupby('title').size()

title
Academy Dinosaur     10
Ace Goldfinger        4
Adaptation Holes      5
Affair Prejudice      5
African Egg           5
                     ..
Young Language        5
Youth Kick            5
Zhivago Core          6
Zoolander Fiction     5
Zorro Ark             3
Length: 997, dtype: int64

In [ ]:
# What the most actors in a film?
actors_df = db['film'].merge(db['film_actor'], left_index=True, right_on='film_id', how='left')
actors_df.groupby('film_id').size().value_counts()

5     195
6     150
4     137
7     119
3     119
8      90
2      69
9      49
1      24
10     21
11     14
13      6
12      6
15      1
dtype: int64

In [ ]:
# What was the film?
actors_df.groupby('film_id').size().sort_values()[-1:]

film_id
508    15
dtype: int64

In [ ]:
db['film'].loc[508]

title                                                Lambs Cincinatti
description         A Insightful Story of a Man And a Feminist who...
release_year                                                     2006
language_id                                                         1
rental_duration                                                     6
rental_rate                                                      4.99
length                                                            144
replacement_cost                                                18.99
rating                                                          PG-13
last_update                                   2013-05-26 14:50:58.951
special_features                       {Trailers,"Behind the Scenes"}
fulltext            'australia':18 'cincinatti':2 'compos':16 'fem...
Name: 508, dtype: object

In [ ]:
## Who acted in the film?
actor_film_df = actor_film_df.reset_index()
actor_film_df[actor_film_df.film_id == 508]

,actor_id,film_id,first_name,last_name,last_update_x,last_update_y,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext
3062,28,508,Woody,Hoffman,2013-05-26 14:47:57.62,2006-02-15 10:05:03,Lambs Cincinatti,A Insightful Story of a Man And a Feminist who...,2006.0,1,6.0,4.99,144.0,18.99,PG-13,2013-05-26 14:50:58.951,"{Trailers,""Behind the Scenes""}",'australia':18 'cincinatti':2 'compos':16 'fem...
3063,37,508,Val,Bolger,2013-05-26 14:47:57.62,2006-02-15 10:05:03,Lambs Cincinatti,A Insightful Story of a Man And a Feminist who...,2006.0,1,6.0,4.99,144.0,18.99,PG-13,2013-05-26 14:50:58.951,"{Trailers,""Behind the Scenes""}",'australia':18 'cincinatti':2 'compos':16 'fem...
3064,45,508,Reese,Kilmer,2013-05-26 14:47:57.62,2006-02-15 10:05:03,Lambs Cincinatti,A Insightful Story of a Man And a Feminist who...,2006.0,1,6.0,4.99,144.0,18.99,PG-13,2013-05-26 14:50:58.951,"{Trailers,""Behind the Scenes""}",'australia':18 'cincinatti':2 'compos':16 'fem...
3065,47,508,Julia,Barrymore,2013-05-26 14:47:57.62,2006-02-15 10:05:03,Lambs Cincinatti,A Insightful Story of a Man And a Feminist who...,2006.0,1,6.0,4.99,144.0,18.99,PG-13,2013-05-26 14:50:58.951,"{Trailers,""Behind the Scenes""}",'australia':18 'cincinatti':2 'compos':16 'fem...
3066,53,508,Mena,Temple,2013-05-26 14:47:57.62,2006-02-15 10:05:03,Lambs Cincinatti,A Insightful Story of a Man And a Feminist who...,2006.0,1,6.0,4.99,144.0,18.99,PG-13,2013-05-26 14:50:58.951,"{Trailers,""Behind the Scenes""}",'australia':18 'cincinatti':2 'compos':16 'fem...
3067,61,508,Christian,Neeson,2013-05-26 14:47:57.62,2006-02-15 10:05:03,Lambs Cincinatti,A Insightful Story of a Man And a Feminist who...,2006.0,1,6.0,4.99,144.0,18.99,PG-13,2013-05-26 14:50:58.951,"{Trailers,""Behind the Scenes""}",'australia':18 'cincinatti':2 'compos':16 'fem...
3068,75,508,Burt,Posey,2013-05-26 14:47:57.62,2006-02-15 10:05:03,Lambs Cincinatti,A Insightful Story of a Man And a Feminist who...,2006.0,1,6.0,4.99,144.0,18.99,PG-13,2013-05-26 14:50:58.951,"{Trailers,""Behind the Scenes""}",'australia':18 'cincinatti':2 'compos':16 'fem...
3069,81,508,Scarlett,Damon,2013-05-26 14:47:57.62,2006-02-15 10:05:03,Lambs Cincinatti,A Insightful Story of a Man And a Feminist who...,2006.0,1,6.0,4.99,144.0,18.99,PG-13,2013-05-26 14:50:58.951,"{Trailers,""Behind the Scenes""}",'australia':18 'cincinatti':2 'compos':16 'fem...
3070,102,508,Walter,Torn,2013-05-26 14:47:57.62,2006-02-15 10:05:03,Lambs Cincinatti,A Insightful Story of a Man And a Feminist who...,2006.0,1,6.0,4.99,144.0,18.99,PG-13,2013-05-26 14:50:58.951,"{Trailers,""Behind the Scenes""}",'australia':18 'cincinatti':2 'compos':16 'fem...
3071,111,508,Cameron,Zellweger,2013-05-26 14:47:57.62,2006-02-15 10:05:03,Lambs Cincinatti,A Insightful Story of a Man And a Feminist who...,2006.0,1,6.0,4.99,144.0,18.99,PG-13,2013-05-26 14:50:58.951,"{Trailers,""Behind the Scenes""}",'australia':18 'cincinatti':2 'compos':16 'fem...


# How long have you had that rental?
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html

In [ ]:
# Look at the row with index 14497
db['rental'].loc[14497]

rental_id                     14503
rental_date     2006-02-14 15:16:03
inventory_id                   2144
customer_id                     188
return_date                      \N
staff_id                          1
last_update     2006-02-16 02:30:53
Name: 14497, dtype: object

In [ ]:
# How many rentals are there?
len(db['rental'])

16044

In [ ]:
# What is the data type of rental table
db['rental'].dtypes

rental_id        int64
rental_date     object
inventory_id     int64
customer_id      int64
return_date     object
staff_id         int64
last_update     object
dtype: object

In [ ]:
# Convert rental date to datetime (from string)
db['rental']['rental_date'] = pd.to_datetime(db['rental']['rental_date'], errors='coerce')

In [ ]:
# Convert return date to datatime (from string) ... Hint (look at the errors parameter)
db['rental']['return_date'] = pd.to_datetime(db['rental']['return_date'], errors='coerce')

In [ ]:
# Add a column to the rental table noting the different in times
db['rental']['duration'] = db['rental']['return_date'] - db['rental']['rental_date']

In [ ]:
# View the rental table
db['rental']

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update,duration
0,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-16 02:30:53,3 days 20:46:00
1,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-16 02:30:53,7 days 23:09:00
2,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-16 02:30:53,9 days 02:39:00
3,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-16 02:30:53,8 days 05:28:00
4,6,2005-05-24 23:08:07,2792,549,2005-05-27 01:32:07,1,2006-02-16 02:30:53,2 days 02:24:00
...,...,...,...,...,...,...,...,...
16039,16046,2005-08-23 22:26:47,4364,74,2005-08-27 18:02:47,2,2006-02-16 02:30:53,3 days 19:36:00
16040,16047,2005-08-23 22:42:48,2088,114,2005-08-25 02:48:48,2,2006-02-16 02:30:53,1 days 04:06:00
16041,16048,2005-08-23 22:43:07,2019,103,2005-08-31 21:33:07,1,2006-02-16 02:30:53,7 days 22:50:00
16042,16049,2005-08-23 22:50:12,2666,393,2005-08-30 01:01:12,2,2006-02-16 02:30:53,6 days 02:11:00


In [ ]:
# What's the average rental duration
db['rental']['duration'].mean()

Timedelta('5 days 00:36:28.541706071')